In [1]:
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import numpy as np

In [2]:
train = pd.read_csv("train_data.csv")
test = pd.read_csv("test_data.csv")
staions = pd.read_csv("station_list.csv")

In [3]:
train.head()

,id,name,host_id,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,last_review,reviews_per_month,availability_365,y
0,1,KiyosumiShirakawa 3min|★SkyTree★|WIFI|Max4|Tre...,242899459,Koto Ku,35.68185,139.80310,Entire home/apt,1,55,2020-04-25,2.21,173,12008
1,2,Downtown Tokyo Iriya next to Ueno,308879948,Taito Ku,35.72063,139.78536,Entire home/apt,6,72,2020-03-25,2.11,9,6667
2,3,"Japan Style,Private,Affordable,4min to Sta.",300877823,Katsushika Ku,35.74723,139.82349,Entire home/apt,1,18,2020-03-23,3.46,288,9923
3,4,4 min to Shinjuku Sta. by train / 2 ppl / Wi-fi,236935461,Shibuya Ku,35.68456,139.68077,Entire home/apt,1,2,2020-04-02,1.76,87,8109
4,5,LICENSED SHINJUKU HOUSE: Heart of the action!,243408889,Shinjuku Ku,35.69840,139.70467,Entire home/apt,1,86,2020-01-30,2.00,156,100390


In [4]:
#欠損値削除
train["reviews_per_month"] = train["reviews_per_month"].fillna(0)
train["last_review"] = train["last_review"].fillna("1900-01-01")
test["reviews_per_month"] = test["reviews_per_month"].fillna(0)

In [5]:
id = test["id"]
train.drop("id",axis=1,inplace=True)
test.drop("id",axis=1,inplace=True)

In [6]:
int_columns = train.select_dtypes(exclude="object").columns
X_train,y_train = train[int_columns].drop("y",axis=1),train["y"]
X_test = test.select_dtypes(exclude="object")

In [7]:
X_test.isnull().sum()

host_id              0
latitude             0
longitude            0
minimum_nights       0
number_of_reviews    0
reviews_per_month    0
availability_365     0
dtype: int64

In [8]:
model = RFR()

In [9]:
#定義したX_train,y_trainを入れる
scores = []
kf = KFold(n_splits=5, shuffle=True, random_state=1)
for tr_idx, va_idx in kf.split(X_train):
    tr_x, va_x = X_train.iloc[tr_idx], X_train.iloc[va_idx]
    tr_y, va_y = y_train.iloc[tr_idx], y_train.iloc[va_idx]
    model.fit(tr_x, tr_y)
    va_pred = model.predict(va_x)
    rmsle = np.sqrt(mean_squared_error(np.log1p(va_y), np.log1p(va_pred)))
    scores.append(rmsle)
    print(rmsle)

# クロスバリデーションの平均のスコアを出力する
print(f'RMSLE: {np.mean(scores):.4f}')

0.8291748766107382
0.8258550382547617
0.7628452234007974
0.7983415161963294
0.8127927854503838
RMSLE: 0.8058


In [10]:
model.fit(X_train,y_train)

RandomForestRegressor()

In [11]:
y_pred = model.predict(X_test)

In [13]:
submission = pd.DataFrame({"id":id,"y":y_pred})
submission

,id,y
0,1,17326.23
1,2,32887.02
2,3,20259.81
3,4,41819.15
4,5,21035.46
...,...,...
4991,4992,16086.25
4992,4993,18978.33
4993,4994,19914.92
4994,4995,18706.53


In [14]:
submission.to_csv("submission.csv",index=False)